In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0`
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .config("spark.sql.shuffle.partitions", 4)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/06/02 22:04:54 INFO SparkContext: Running Spark version 3.2.0
22/06/02 22:04:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/02 22:04:54 INFO ResourceUtils: ==============================================================
22/06/02 22:04:54 INFO ResourceUtils: No custom resources configured for spark.driver.
22/06/02 22:04:54 INFO ResourceUtils: ==============================================================
22/06/02 22:04:54 INFO SparkContext: Submitted application: 5679b4dc-1162-44c4-9abd-658cc986f1c5
22/06/02 22:04:54 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/06/0

import $ivy.$                                  

import $ivy.$                               


import org.apache.spark.sql.{SparkSession, DataFrame}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._

import org.apache.spark.sql.expressions.Window


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@485314f2
import spark.implicits._



In [2]:
def constructorQuery(races: DataFrame, 
                     constructor_standings: DataFrame, 
                     constructors: DataFrame
                    ): DataFrame = {
    

    val lastRace = Window.partitionBy("year")

    val lastRaces = races
        .where(col("year") >= 1990 && col("year") <= 1999)
        .withColumn("round", col("round").cast(IntegerType))
        .withColumn("max", max(col("round")).over(lastRace))
        .where(col("round") === col("max"))
        .select("raceId", "year")
    
    val constructorMap = constructors
        .select("constructorId", "name")
    
    val constructorWindow = Window.partitionBy("constructorId")

    constructor_standings
        .join(lastRaces, Seq("raceId"), "right")
        .where(col("position") === 1)
        .withColumn("totalChampWins", 
                    count(col("constructorId")).over(constructorWindow))
        .withColumn("totalRaceWins", 
                    sum(col("wins")).over(constructorWindow))
        .dropDuplicates("constructorId")
        .join(constructorMap, "constructorId")
        .select("totalChampWins", "totalRaceWins", "name")
        .orderBy(col("totalChampWins").desc, col("totalRaceWins").desc)
    
}

defined function constructorQuery

In [3]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")

val constructor_standings = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/constructor_standings.csv")

val constructors = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/constructors.csv")

val results = constructorQuery(races, constructor_standings, constructors)

results.show()

+--------------+-------------+--------+
|totalChampWins|totalRaceWins|    name|
+--------------+-------------+--------+
|             5|         47.0|Williams|
|             3|         23.0| McLaren|
|             1|         11.0|Benetton|
|             1|          6.0| Ferrari|
+--------------+-------------+--------+



races: DataFrame = [raceId: string, year: string ... 6 more fields]
constructor_standings: DataFrame = [constructorStandingsId: string, raceId: string ... 5 more fields]
constructors: DataFrame = [constructorId: string, constructorRef: string ... 3 more fields]
results: DataFrame = [totalChampWins: bigint, totalRaceWins: double ... 1 more field]

In [4]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells


In [5]:
val constructors = results
    .select("name")
    .as[String]
    .collect()
    .toList

val raceWins = results
    .select("totalRaceWins")
    .as[String]
    .collect()
    .toList

val data = Seq(Bar(constructors, raceWins))

val layout = Layout(barmode = BarMode.Group)

plot(data, layout)

constructors: List[String] = List("Williams", "McLaren", "Benetton", "Ferrari")
raceWins: List[String] = List("47.0", "23.0", "11.0", "6.0")
data: Seq[Bar] = List(
  Bar(
...
layout: Layout = Layout(
  None,
...
res4_4: String = "plot-c3662ce2-b24f-496e-baa3-98477dd5bba2"